<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/Korean_QA_on_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a QA System with BERT on Wikipedia

https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

위의 내용을 한국어 QA로 변경

지식 base = wiki+네이버(view,kin,news)

In [1]:
!pip install transformers==3
!pip install wikipedia
!pip install sentence-transformers

     |████████████████████████████████| 757kB 7.7MB/s 
     |████████████████████████████████| 3.0MB 16.1MB/s 
     |████████████████████████████████| 890kB 41.7MB/s 
     |████████████████████████████████| 1.1MB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1203de4073f77cf0914ecbaa25255c18e49c03ea7111be3e4359b28a8b40f67e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=9a4f217a7f7b28db7049d7d5336ad070e661feed5cb5dce2ab9269bb291169eb
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 1.3MB 10.9MB/s 
     |████████████████████████████████| 2.9MB 34.0MB/s 
     |██████████████

In [50]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict

class DocumentReader:
    def __init__(self, pretrained_model_name_or_path=''):
        self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True

    def chunkify(self):
        """ 
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model. 

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)
            
            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input

    def get_answer(self):
        answer = ''
        if self.chunked:
            
            for k, chunk in self.inputs.items():
                answer_start_scores, answer_end_scores = self.model(**chunk)

                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1

                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans.startswith(('[CLS]','[SEP]',' ','°')):
                    #raise Exception('No Answer')
                    pass
                else:
                    answer += ans + ', '
                    #break
        else:
            answer_start_scores, answer_end_scores = self.model(**self.inputs)

            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
        
            answer = self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])
        if answer in ['',' ','  ']:
          raise Exception('No Answer')                    
        return answer
        
    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

##한국어 SQUAD
KorQuAD 2.0 사용 https://korquad.github.io/

##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

**Available Pre-trained Models**

*   **distiluse-base-multilingual-cased**: Supported languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish. Model is based on DistilBERT-multi-lingual.
*   **xlm-r-base-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 81.47
*   **xlm-r-large-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 84.05 --> 이거 사용!


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md



In [3]:
from sentence_transformers import SentenceTransformer

SQUAD_MODEL = "monologg/koelectra-base-v3-finetuned-korquad"
STS_MODEL = "xlm-r-large-en-ko-nli-ststb"
reader = DocumentReader(SQUAD_MODEL) 
embedder = SentenceTransformer(STS_MODEL)

100%|██████████| 1.80G/1.80G [01:58<00:00, 15.2MB/s]


In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict
import scipy
import requests
from bs4 import BeautifulSoup

class Korean_QA_on_Wiki:
  def __init__(self, document_reader,sentence_embedder):
    self.reader = document_reader
    self.embedder = sentence_embedder
    wiki.set_lang('ko')

  def __search_from_wiki(self,question,max_rank):
    results = wiki.search(question,results=max_rank)
    contents = []
    for result in results:
      try:
        page = wiki.page(result)
        #print(f"Top wiki result: {page}")
        text = page.content
        contents.append((text,page))
      except Exception as ex:
        print(ex)
    return contents

  def __search_from_naver(self,question,max_rank):
    contents = []
    url = 'https://search.naver.com/search.naver'
    for w in ['view','kin','news']:
      params = {'query': question,'where': w,}
      response = requests.get(url, params=params)
      html = response.text
      #뷰티풀소프의 인자값 지정
      soup = BeautifulSoup(html, 'html.parser')
      #쪼개기
      #title_list = soup.find_all('a', href=True)
      title_list = soup.select('.api_txt_lines')
      #print(title_list)
      tmp = []
      for tag in title_list:
        tmp.append(tag.text)
      contents.append((''.join(tmp),url + '?where='+w))
      tmp.clear()
    #print(contents)      
    return contents



  def question(self, questions, max_rank = 10):
    answers = {}
    for question in questions:
        print(f"Question: {question}")
        contents = []
        contents.extend(self.__search_from_wiki(question,max_rank))
        contents.extend(self.__search_from_naver(question,max_rank))
        #print(len(contents))
        corpus_embeddings = self.embedder.encode([a for (a,b) in contents],show_progress_bar=False) 
        query_embeddings = self.embedder.encode([question])
        distances = scipy.spatial.distance.cdist(query_embeddings, corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        answer_list = []
        for idx, distance in results:
            text = contents[idx][0]
            #print(text)
            try:
                self.reader.tokenize(question, text)
                t = (self.reader.get_answer(),contents[idx][1])
                print(f"Answer: {t[0]}", f" from {t[1]}")
                answer_list.append(t)
                
            except Exception as ex:
                pass    

        answers[question] = answer_list
        print(' ')
    return answers


In [52]:
kqaw = Korean_QA_on_Wiki(reader,embedder)

In [53]:
answers = kqaw.question(["북한에서 실질적인 권력자는 누구인가?",
                           "세계에서 가장 넓은 호수는?",
                           "오로라가 가장 잘 보이는 곳은?",
                           "심장이 죄어오듯이 아프면 의심되는 병은 무엇인가?",
                           "항문에서 피가 나는 병은 무엇인가?",
                           "김재규는 박정희를 왜 죽였는가?",
                           "케네디를 죽인 암살범은 누구인가?",
                           "술 취하지 않는 방법은?",
                           "사람을 사랑해서 생기는 병은?",
                           "부모는 자식을 왜 사랑하는가?",
                           "나의 와이프는 나를 사랑하는가?",
                           "신은 존재 하는가?",
                           "사람의 인생에서 가장 소중한 것은 무엇인가?",
                           "바람난 여자는 다시 돌아올 수 있는가?",
                           "위가 쓰리고 아플 때 어떤 약을 복용해야 하는가?",
                           "눈알이 빠지면 어떻게 되는가?"])

Question: 북한에서 실질적인 권력자는 누구인가?
Answer: 누군가요 ? . . . 김정은은 그냥 얼굴마담인가요 ? 그럼 실질적인 북한권력의 주인이 누군가요 ? 김일성의 모습과 말투를 따라하는 김정은은 그냥 권력의 끝자락을 쥐고 있는 젊은이일 뿐인가요 ? 지금 북한의 권력구조에 대해 정통한 분 . . . 알려주세요 . . 북한의 실질적인 권력자가 누구인지 ? ? 언론 이나 관계기관의 발표 등을 종합해 볼 때 현재로서는 북한의 최고 권력자는 김정일의 대를 이은 김정은이라고 봐야 할 것입니다 . 김정은은 김정일 사망 후 권력을 인계받은 수 8개월동안 권력 안정화 작업을 진행중에 있으며 나름대로 성과를 거두고 있는것으로 알려지고 있습니다 . 그 . . . 6 . 25전쟁과 북한에 관한 건데요 . . . . 그리고 서울1945를 보면서 그 누구더라 그사람이 죽자 최운혁이 괴로워하면서 북으로 넘어갔는데 아 몽양 은 생각나는데 하여튼  from https://search.naver.com/search.naver?where=kin
Answer: 김정은  from https://search.naver.com/search.naver?where=news
Answer: 박정희  from <WikipediaPage '박정희'>
Answer: 시진핑  from https://search.naver.com/search.naver?where=view
 
Question: 세계에서 가장 넓은 호수는?
Answer: 카스피해  from https://search.naver.com/search.naver?where=kin
Answer: 바이칼 호  from https://search.naver.com/search.naver?where=view
Answer: 티베트 자치구 남초 호  from <WikipediaPage '염호'>
Answer: 카스피 해  from <WikipediaPage '호수'>
Answer: 보스토크호  from <WikipediaPage '보스토크호'>
Ans

In [54]:
answers = kqaw.question(["아브라함은 자식이 몇명인가?"])

Question: 아브라함은 자식이 몇명인가?
Answer: 8명  from https://search.naver.com/search.naver?where=view
Answer: 몇 명  from https://search.naver.com/search.naver?where=news
Answer: 6명  from <WikipediaPage '아브라함'>
Answer: 13명  from <WikipediaPage '소저너 트루스'>
 


In [60]:
answers = kqaw.question(["아시아나항공은 어디에 매각될 것인가?",
                         "박세창은 어느 회사의 사장인가?"])

Question: 아시아나항공은 어디에 매각될 것인가?
Answer: 대한항공  from https://search.naver.com/search.naver?where=news
 
Question: 박세창은 어느 회사의 사장인가?
Answer: 아시아나IDT  from https://search.naver.com/search.naver?where=view
Answer: 아시아나IDT  from https://search.naver.com/search.naver?where=news
Answer: 금호석유화학  from <WikipediaPage '금호석유화학'>
 


In [62]:
answers = kqaw.question(["아시아나항공 사장의 이름은?",
                         "금호건설의 사장은 누구인가?"])

Question: 아시아나항공 사장의 이름은?
Answer: 한창수  from https://search.naver.com/search.naver?where=view
Answer: 박정구  from <WikipediaPage '박정구'>
 
Question: 금호건설의 사장은 누구인가?
Answer: 박정구  from https://search.naver.com/search.naver?where=view
 


In [63]:
answers = kqaw.question(["아파트 값은 계속 오를 것인가?",
                         "코로나는 언제 종식 될 것인가?"])

Question: 아파트 값은 계속 오를 것인가?
 
Question: 코로나는 언제 종식 될 것인가?
Answer: 2021년 9월 중순  from https://search.naver.com/search.naver?where=kin
 


In [66]:
answers = kqaw.question(["단백질의 화학식 구성은 어떻게 되는가?"])

Question: 단백질의 화학식 구성은 어떻게 되는가?
Answer: 아미노산  from <WikipediaPage '생화학'>
Answer: NH2CHRnCOOH ) n  from <WikipediaPage '단백질'>
Answer: C5H10O4  from <WikipediaPage '탄수화물'>
Answer: ##NH2CHRnCOOH  from <WikipediaPage '아미노산'>
Answer: C6H10O5  from <WikipediaPage '다당류'>
Answer: C5H10O5  from <WikipediaPage '아라비노스'>
Answer: Geraniol  from https://search.naver.com/search.naver?where=view
Answer: HO2CCH2NH2  from <WikipediaPage '글라이신'>
Answer: HO2CCH ( NH2 ) CH2OH  from <WikipediaPage '세린'>
 


In [67]:
answers = kqaw.question(["우리나라 특허의 권리보장 기간은 몇년인가?"])

Question: 우리나라 특허의 권리보장 기간은 몇년인가?
Answer: 20년  from https://search.naver.com/search.naver?where=kin
Answer: 20년  from https://search.naver.com/search.naver?where=view
 


In [68]:
answers = kqaw.question(["발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?"])

Question: 발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?
Answer: 알레르기 비염  from https://search.naver.com/search.naver?where=view
Answer: 코로나 19  from https://search.naver.com/search.naver?where=kin
Answer: 호흡곤란 , 후각상실 , 미각상실  from <WikipediaPage '코로나19 범유행'>
 


In [72]:
answers = kqaw.question(["흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?"])

Question: 흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?
Answer: 폐암  from https://search.naver.com/search.naver?where=view
 


In [73]:
answers = kqaw.question(["똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?"])

Question: 똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?
Answer: 치열  from https://search.naver.com/search.naver?where=kin
Answer: 호흡기질환  from https://search.naver.com/search.naver?where=view
 


In [74]:
answers = kqaw.question(["신경망 알고리즘의 활성함수에는 어떤 것이 있나요?"])

Question: 신경망 알고리즘의 활성함수에는 어떤 것이 있나요?
Answer: 입력 이미지의 픽셀  from <WikipediaPage '인공 신경망'>
Answer: 시그모이드  from https://search.naver.com/search.naver?where=kin
Answer: 노드의 가중치와 입력치  from <WikipediaPage '퍼셉트론'>
 


In [77]:
answers = kqaw.question(["현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?"])

Question: 현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?
Answer: 파이썬  from https://search.naver.com/search.naver?where=news
Answer: 인간의 뇌  from https://search.naver.com/search.naver?where=kin
 


In [81]:
answers = kqaw.question(["이세돌을 이긴 것은 무엇입니까?"])

Question: 이세돌을 이긴 것은 무엇입니까?
Answer: 알파고  from https://search.naver.com/search.naver?where=view
Answer: 알파고  from https://search.naver.com/search.naver?where=kin
 


In [82]:
answers = kqaw.question(["피타고라스는 어느 나라 사람인가?"])

Question: 피타고라스는 어느 나라 사람인가?
Answer: 그리스  from https://search.naver.com/search.naver?where=kin
Answer: 이탈리아  from <WikipediaPage '고대 그리스 철학'>
Answer: 그리스  from https://search.naver.com/search.naver?where=news
Answer: 서양의 경우 , 고대 이집트  from <WikipediaPage '윤회'>
 


In [93]:
answers = kqaw.question(["아이즈원 멤버 인원수는?"])

Question: 아이즈원 멤버 인원수는?
Answer: 12명  from https://search.naver.com/search.naver?where=news
Answer: 8명  from https://search.naver.com/search.naver?where=kin
Answer: 300명  from <WikipediaPage 'HKT48'>
 


In [91]:
answers = kqaw.question(["트와이스 중에 가장 인기 있는 사람은?"])

Question: 트와이스 중에 가장 인기 있는 사람은?
Answer: 사나  from https://search.naver.com/search.naver?where=view
Answer: 박진영  from <WikipediaPage '박진영'>
Answer: 지효  from <WikipediaPage '청하 (가수)'>
Answer: 공승연  from <WikipediaPage '공승연'>
 


In [95]:
answers = kqaw.question(["벤츠 자동차가 처음 발명된 년도는?"])

Question: 벤츠 자동차가 처음 발명된 년도는?
Answer: 1883년  from https://search.naver.com/search.naver?where=kin
Answer: 1831년  from <WikipediaPage '지크프리트 마르쿠스'>
Answer: 1886년  from <WikipediaPage '자동차'>
Answer: 1885년  from https://search.naver.com/search.naver?where=view
Answer: 1886년  from <WikipediaPage '만하임'>
 


In [101]:
answers = kqaw.question(["교통사고 대비를 위해 들어야 하는 보험은 무엇인가?"])

Question: 교통사고 대비를 위해 들어야 하는 보험은 무엇인가?
Answer: 자동차보험  from https://search.naver.com/search.naver?where=news
 


In [104]:
answers = kqaw.question(["너의 이름은 무엇이니?"])

Question: 너의 이름은 무엇이니?
Answer: 레아  from https://search.naver.com/search.naver?where=news
 


In [105]:
answers = kqaw.question(["너는 남자니 여자니?"])

Question: 너는 남자니 여자니?
 
